In [1]:
!date

Tue Apr 27 11:45:40 PDT 2021


# Install packages and download data

In [7]:
import sys
COLAB = "google.colab" in sys.modules


In [1]:
if True:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 165 (delta 53), reused 121 (delta 23), pack-reused 0
Receiving objects: 100% (165/165), 59.24 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [2]:
!pip --quiet install CITE-seq-Count==1.4.4

In [4]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6 \
    CITE-seq-Count==1.4.4

In [3]:
if Colab:
    # download the relevant data
    !wget -O cellhash_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/mpv37niu7kxbrx3gb4mfbun3t9zqv3ad.gz \
    2>&1
    !tar -xf cellhash_fastqs.tar.gz

cellhash_fastqs.tar 100%[===================>]   6.30G  17.0MB/s    in 6m 50s  


# Preprocess with kb

In [4]:
%%time
!INDEX="BMGP_2020/references/cellhash/kite/features.idx" && \
F2B="BMGP_2020/references/cellhash/kite/f2b.txt" && \
FASTA="BMGP_2020/references/cellhash/kite/features.fa" && \
FB="BMGP_2020/references/cellhash/kite/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-05-18 12:20:15,395]    INFO Generating mismatch FASTA at BMGP_2020/references/cellhash/kite/features.fa
[2021-05-18 12:20:15,401]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/cellhash/kite/f2b.txt
[2021-05-18 12:20:15,404]    INFO Skipping kallisto index because BMGP_2020/references/cellhash/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 24 ms, sys: 5.78 ms, total: 29.8 ms
Wall time: 1.66 s


In [7]:
%%bash
FB="BMGP_2020/references/cellhash/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/cellhash/kite/features.idx" && \
F2B="BMGP_2020/references/cellhash/kite/f2b.txt" && \
FASTA="BMGP_2020/references/cellhash/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
/usr/bin/time --output=memtime_0.log -v kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-06-30 14:33:42,830]    INFO [count] Using index BMGP_2020/references/cellhash/kite/features.idx to generate BUS file to kite_out from
[2021-06-30 14:33:42,830]    INFO [count]         fastqs/SRR8281307_R1.fastq.gz
[2021-06-30 14:33:42,830]    INFO [count]         fastqs/SRR8281307_R2.fastq.gz
[2021-06-30 14:36:19,376]    INFO [count] Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-06-30 14:36:39,237]    INFO [count] Whitelist not provided
[2021-06-30 14:36:39,237]    INFO [count] Copying pre-packaged 10XV2 whitelist to kite_out
[2021-06-30 14:36:39,330]    INFO [count] Inspecting BUS file kite_out/tmp/output.s.bus
[2021-06-30 14:36:43,297]    INFO [count] Correcting BUS records in kite_out/tmp/output.s.bus to kite_out/tmp/output.s.c.bus with whitelist kite_out/10x_version2_whitelist.txt
[2021-06-30 14:36:47,903]    INFO [count] Sorting BUS file kite_out/tmp/output.s.c.bus to kite_out/output.unfiltered.bus
[2021-06-30 14:36:56,207]    INFO [count] Generatin

# Preprocess with cite-seq-count

In [3]:
!gunzip BMGP_2020/references/whitelist_10xv2.txt.gz

In [8]:
!R1="fastqs/SRR8281307_R1.fastq.gz" && \
R2="fastqs/SRR8281307_R2.fastq.gz" && \
FB="BMGP_2020/references/cellhash/cite_seq_barcodes.csv" && \
WL="BMGP_2020/references/whitelist_10xv2.txt" && \
/usr/bin/time --output=memtime_1.log -v CITE-seq-Count -R1 $R1  -R2 $R2 --no_umi_correction -t $FB -wl $WL -cbf 1 -cbl 16 -umif 17 -umil 26 -cells 65000 -o ./cite_seq_out

Loading whitelist
[WARNING] Read1 length is 29bp but you are using 26bp for Cell and UMI barcodes combined.
This might lead to wrong cell attribution and skewed umi counts.

Counting number of reads
Started mapping
Processing 74,219,921 reads
CITE-seq-Count is running with 48 cores.
Processed 1,000,000 reads in 13.2 seconds. Total reads: 1,000,000 in child 18827
Mapping done for process 18827. Processed 1,546,248 reads
Processed 1,000,000 reads in 19.55 seconds. Total reads: 1,000,000 in child 18828
Mapping done for process 18828. Processed 1,546,248 reads
Processed 1,000,000 reads in 27.81 seconds. Total reads: 1,000,000 in child 18829
Mapping done for process 18829. Processed 1,546,248 reads
Mapping done for process 18830. Processed 1,546,248 reads
Processed 1,000,000 reads in 45.97 seconds. Total reads: 1,000,000 in child 18831
Mapping done for process 18831. Processed 1,546,248 reads
Processed 1,000,000 reads in 51.63 seconds. Total reads: 1,000,000 in child 18832
Mapping done for 